In [ ]:
import pandas as pd

In [ ]:
df_bagerhat = pd.read_csv('Bagerhat_historical_weather_1980_2024.csv')

In [ ]:
shape = df_bagerhat.shape
print(f"Dataset contains {shape[0]} rows and {shape[1]} columns.")

Dataset contains 16437 rows and 15 columns.


In [ ]:
df_bagerhat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16437 entries, 0 to 16436
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   district                       16437 non-null  object 
 1   day                            16437 non-null  int64  
 2   month                          16437 non-null  int64  
 3   year                           16437 non-null  int64  
 4   temperature(degree C)          10478 non-null  float64
 5   feels_like(degree C)           10478 non-null  float64
 6   max_temperature(degree C)      14991 non-null  float64
 7   minimum_temperature(degree C)  14991 non-null  float64
 8   humidity                       10478 non-null  float64
 9   precipitation                  5479 non-null   float64
 10  windspeed                      11924 non-null  float64
 11  atmospheric_pressure           7115 non-null   float64
 12  UV                             4038 non-null  

In [ ]:
import pandas as pd

df = df_bagerhat.copy()

# Build datetime index
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df.sort_values('date').set_index('date')

# Define feature groups
core_cols = [
    'temperature(degree C)', 'feels_like(degree C)',
    'max_temperature(degree C)', 'minimum_temperature(degree C)',
    'humidity', 'windspeed', 'atmospheric_pressure', 'dew_point'
]

optional_cols = ['precipitation', 'UV', 'solar_radiation']

# ---- Step 1: Handle missing gaps for core variables ----
mask = df[core_cols].isna().any(axis=1)
group = (mask != mask.shift()).cumsum()
gap_lengths = mask.groupby(group).transform('sum')

# Drop long missing sequences (≥6 days)
df = df[~((mask) & (gap_lengths >= 6))]

# ---- Step 2: Interpolate ----
# Interpolate core features with time-based interpolation
df[core_cols] = df[core_cols].interpolate(method='time')

# Interpolate optional ones separately, if they exist
for col in optional_cols:
    if col in df.columns:
        df[col] = df[col].interpolate(method='time', limit_direction='both')

# ---- Step 3: Drop rows only if core variables still missing ----
df = df.dropna(subset=core_cols)

# ---- Step 4: Summary ----
print(f"After cleaning, dataset contains {df.shape[0]} rows and {df.shape[1]} columns out of {df_bagerhat.shape[0]} rows. (taking precipitation as optional)")
missing_ratio = df.isna().mean().round(3) * 100
print("\nRemaining missing % per column:")
print(missing_ratio)


After cleaning, dataset contains 7303 rows and 15 columns out of 16437 rows. (taking precipitation as optional)

Remaining missing % per column:
district                         0.0
day                              0.0
month                            0.0
year                             0.0
temperature(degree C)            0.0
feels_like(degree C)             0.0
max_temperature(degree C)        0.0
minimum_temperature(degree C)    0.0
humidity                         0.0
precipitation                    0.0
windspeed                        0.0
atmospheric_pressure             0.0
UV                               0.0
solar_radiation                  0.0
dew_point                        0.0
dtype: float64


In [ ]:
long_gaps = gap_lengths[mask & (gap_lengths >= 6)]
print(f"Dropped {long_gaps.nunique()} long missing sequences (≥6 days).")

Dropped 48 long missing sequences (≥6 days).


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# assuming your cleaned + interpolated DataFrame is named 'df'
with_interpolation = df.copy()

# ensure datetime column exists (if not already index)
if 'date' not in with_interpolation.columns:
    with_interpolation['date'] = pd.to_datetime(with_interpolation[['year', 'month', 'day']])

# show date range
min_date = with_interpolation['date'].min()
max_date = with_interpolation['date'].max()
print(f"Date range: {min_date.date()} to {max_date.date()}")

# --- Plot 1: Daily Temperature (interpolated) ---
plt.figure(figsize=(12, 4))
plt.plot(with_interpolation['date'], with_interpolation['temperature(degree C)'],
         marker='.', linestyle='-', alpha=0.6, color='tab:blue')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.title('Daily Temperature (with_interpolation)')
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Plot 2: Monthly Average Temperature (interpolated) ---
monthly_avg = with_interpolation.set_index('date')['temperature(degree C)'].resample('M').mean()

plt.figure(figsize=(12, 4))
plt.plot(monthly_avg.index, monthly_avg.values, color='red', linewidth=2)
plt.xlabel('Date')
plt.ylabel('Monthly Average Temperature (°C)')
plt.title('Monthly Average Temperature (with_interpolation)')
plt.grid(True)
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:

df = df_bagerhat.copy()

# Build datetime index
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df.sort_values('date').set_index('date')

# Define feature groups
core_cols = [
    'temperature(degree C)', 'feels_like(degree C)',
    'max_temperature(degree C)', 'minimum_temperature(degree C)',
    'humidity', 'windspeed', 'precipitation','atmospheric_pressure', 'dew_point'
]

optional_cols = [ 'UV', 'solar_radiation']

# ---- Step 1: Handle missing gaps for core variables ----
mask = df[core_cols].isna().any(axis=1)
group = (mask != mask.shift()).cumsum()
gap_lengths = mask.groupby(group).transform('sum')

# Drop long missing sequences (≥6 days)
df = df[~((mask) & (gap_lengths >= 6))]

# ---- Step 2: Interpolate ----
# Interpolate core features with time-based interpolation
df[core_cols] = df[core_cols].interpolate(method='time')

# Interpolate optional ones separately, if they exist
for col in optional_cols:
    if col in df.columns:
        df[col] = df[col].interpolate(method='time', limit_direction='both')

# ---- Step 3: Drop rows only if core variables still missing ----
df = df.dropna(subset=core_cols)

# ---- Step 4: Summary ----
print(f"After cleaning, dataset contains {df.shape[0]} rows and {df.shape[1]} columns out of {df_bagerhat.shape[0]} rows. (taking precipitation as mandatory)")
missing_ratio = df.isna().mean().round(3) * 100
print("\nRemaining missing % per column:")
print(missing_ratio)


After cleaning, dataset contains 5479 rows and 15 columns out of 16437 rows. (taking precipitation as mandatory)

Remaining missing % per column:
district                         0.0
day                              0.0
month                            0.0
year                             0.0
temperature(degree C)            0.0
feels_like(degree C)             0.0
max_temperature(degree C)        0.0
minimum_temperature(degree C)    0.0
humidity                         0.0
precipitation                    0.0
windspeed                        0.0
atmospheric_pressure             0.0
UV                               0.0
solar_radiation                  0.0
dew_point                        0.0
dtype: float64


In [ ]:
long_gaps = gap_lengths[mask & (gap_lengths >= 6)]
print(f"Dropped {long_gaps.nunique()} long missing sequences (≥6 days).")

Dropped 1 long missing sequences (≥6 days).
